<font size="+4" color=purple><u><center>COVID Pre Conditions Analysis</center></u></font>

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of content</h3>

* [Introduction](#intro)
* [Packages and Data](#data)
* [Preprocessing](#preprocessing)
* [1. Analysis of symptoms, entry and death dates](#dates)
* [2. Analysis of Covid Results across gender](#gender)
* [3. Analysis of age across age-frequency, gender](#age)

<a id="intro"></a>
<font size="+2" color="blue"><b>Introduction</b></font><br>

Persons with underlying health conditions or other recognized risk factors for severe outcomes from respiratory infections appear to be at a higher risk for severe disease from COVID-19 than are persons without these condition.

https://www.forbes.com/sites/victoriaforster/2020/03/31/people-with-underlying-health-conditions-are-at-risk-of-more-severe-covid-19new-cdc-report/#251f3ec27462

<a id="data"></a>
<font size="+2" color="blue">Packages and Data</font>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import datetime

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as py
import plotly.express as px

import seaborn as sns
from matplotlib import rcParams
import plotly.graph_objects as go

import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/covid19-patient-precondition-dataset/covid.csv", parse_dates=['entry_date','date_symptoms'], dayfirst=True)
df

<a id="preprocessing"></a>
<font size="+2" color="blue">Preprocessing</font>

In [ ]:
no_icu_data_bool = df['icu'].isin([97, 98, 99])
no_icu_data_bool

icu_data = df[~ no_icu_data_bool]
no_icu_data = df[no_icu_data_bool]
print("{} rows have ICU details ".format(icu_data.shape[0]))
print("Only {}% of given data has ICU details ".format(round((icu_data.shape[0]/ no_icu_data.shape[0])*100)))

In [ ]:
icu_data.sex.replace({1: 'Female', 2: 'Male'}, inplace=True)
icu_data.patient_type.replace({1: 'Outpatient', 2: 'Inpatient'}, inplace=True)
icu_data.intubed.replace({1: 'Yes', 2: 'No',97:'Not Specified', 98:'Not Specified',99:'Not Specified'}, inplace=True)
icu_data.pneumonia.replace({1: 'Yes', 2: 'No', 98:'Not Specified',99:'Not Specified', 97:'Not Specified'}, inplace=True)
icu_data.pregnancy.replace({1: 'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
icu_data.diabetes.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified',99:'Not Specified'}, inplace=True)
icu_data.copd.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified', 99:'Not Specified'}, inplace=True)
icu_data.asthma.replace({1: 'Yes', 2: 'No', 99:'Not Specified',97:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.inmsupr.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified',99:'Not Specified'}, inplace=True)
icu_data.hypertension.replace({1: 'Yes', 2: 'No', 99:'Not Specified',97:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.other_disease.replace({1: 'Yes', 2: 'No', 97:'Not Specified',99:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.cardiovascular.replace({1: 'Yes', 2: 'No', 99:'Not Specified',97:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.obesity.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified',99:'Not Specified'}, inplace=True)
icu_data.renal_chronic.replace({1: 'Yes', 2: 'No', 97:'Not Specified',99:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.tobacco.replace({1: 'Yes', 2: 'No', 97:'Not Specified',99:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.contact_other_covid.replace({1: 'Yes', 2: 'No', 97:'Not Specified',99:'Not Specified',98:'Not Specified'}, inplace=True)
icu_data.covid_res.replace({1: 'Positive', 2: 'Negative', 3:'Awaiting Results'}, inplace=True)
icu_data.icu.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified', 99:'Not Specified'}, inplace=True)

icu_yes = icu_data[icu_data['icu'] == "Yes"]
icu_no = icu_data[icu_data['icu'] != "Yes"]
print("In Patients requiring ICU = {} and not requiring ICU = {} ".format(icu_yes.shape[0],icu_no.shape[0]))
icu_yes.date_died = icu_yes.date_died.replace("9999-99-99",datetime.datetime(1900,1,1))
icu_yes.date_died = pd.to_datetime(icu_yes.date_died, dayfirst=True, errors='coerce')
icu_yes['died'] = np.where(icu_yes['date_died'] == datetime.datetime(1900,1,1),False,True)
icu_yes['entry_symptoms_dates'] = ((icu_yes['entry_date'] - icu_yes['date_symptoms'])/ np.timedelta64(1, 'D')).astype(int)
icu_yes['died_entry_dates'] = ((icu_yes['date_died'] - icu_yes['entry_date'])/ np.timedelta64(1, 'D')).astype(int)
icu_yes['died_symptoms_dates'] = ((icu_yes['date_died'] - icu_yes['date_symptoms'])/ np.timedelta64(1, 'D')).astype(int)
icu_yes['age_freq']=np.where((icu_yes['age'] < 2),'<2',np.where((icu_yes['age'] >= 2) & (icu_yes['age'] <= 12) ,'2-12',np.where((icu_yes['age'] >= 13) & (icu_yes['age'] < 18) ,'13-17',np.where(icu_yes['age']<18,'<18',np.where((icu_yes['age']>17)&(icu_yes['age']<=30),'18-30',
np.where((icu_yes['age']>30)&(icu_yes['age']<=50),'31-50',np.where(icu_yes['age']>70,'70+',
np.where((icu_yes['age']>50)&(icu_yes['age']<=70),'51-70',"Not Specified"))))))))

icu_yes.head(4)

<font size="+2" color="blue">Lung related Infections </font>

As per article - https://coronavirusexplained.ukri.org/en/article/cad0012/

The most important consequence of severe COVID-19 is a reduction in the lung’s ability to transfer oxygen from the air into the blood – leading to low blood oxygen levels (hypoxia). Supportive treatment in hospital is therefore mainly to increase blood oxygen.

Pneumonia
The lungs resemble a sponge with each hole representing an air sac (alveolus) and material delicately holding the air sacs and the blood vessels together. The virus enters the lungs in droplets and infects cells lining the air sacs where it multiplies. This directly damages those cells, but also triggers a response from the immune cells nearby, which is called inflammation. Fluid collects in the lung tissue causing pneumonia. This impairs the transfer of oxygen between air sacs and blood, often causing the patient to experience shortness of breath as the body tries to compensate for the inefficient gas transfer by breathing faster. 

Intubation is a procedure that's used when you can't breathe on your own. Your doctor puts a tube down your throat and into your windpipe to make it easier to get air into and out of your lungs. A machine called a ventilator pumps in air with extra oxygen. Then it helps you breathe out air that’s full of carbon dioxide (CO2). This is called mechanical ventilation. It helps keep your oxygen and CO2 at healthy levels.

<font size="+1" color="blue">Asthma and COVID </font>
https://www.cdc.gov/coronavirus/2019-ncov/need-extra-precautions/asthma.html

People with moderate to severe asthma may be at higher risk of getting very sick from COVID-19.  COVID-19 can affect your respiratory tract (nose, throat, lungs), cause an asthma attack, and possibly lead to pneumonia and acute respiratory disease.



<font size="+1" color="darkred">Points to Note: 
* Task of the NB is to find how many cases may need ICU or per how many inpatients can we expect a demand for ICU.
* In the given data, only 121788 rows (Only 27% of given data) have ICU details - so we will take up these only
* ICU patients are inpatients
* Out of 121788 patients only 10112 needed ICU. (Around 8.3%)
* We will look at what kind of health conditions or diseases led to ICU requirement.
</font>

<a id="dates"></a>
<font size="+2" color="blue">Analysis of symptoms, entry and death dates</font>

In [ ]:
def plot_dates(column_name,title):             
    monthly_df = icu_yes.groupby([icu_yes[column_name].dt.to_period("M").astype(str),'sex'])['id'].agg('count').to_frame(name='count').reset_index()   
    if column_name == 'date_died': # removing the first 2 rows because of some faulty data
        monthly_df = monthly_df[2:]        
   
    fig = px.bar(monthly_df, x=column_name, y="count",color="sex",title="Patients realized symptoms")        
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_layout(title_text='{} (Jan-Jun 2020)'.format(title), title_x=0.5,showlegend=True)
    fig.show()
plot_dates('date_symptoms','Patient symptoms arisen')
plot_dates('entry_date','Patient Admissions')
plot_dates('date_died','Patient deaths')

<font size="+1" color="darkred">Insights: 
* Months of January, February saw cases of symptoms and admission entry to hospitals. 
* From March onwards, patients deaths have been occurring. 
* There is slight dip in June in symptoms/admissions and death cases.</font>

In [ ]:
symptoms_by_date = icu_yes.groupby('date_symptoms')['id'].agg('count').to_frame(name='count')
deaths_by_date = icu_yes.groupby('date_died')['id'].agg('count').to_frame(name='count')
entry_by_date = icu_yes.groupby('entry_date')['id'].agg('count').to_frame(name='count')

deaths_by_date = deaths_by_date[1:]
colort = '#456213'
color1 = '#9467bd'
color2 = '#2367ff'
color3 = '#15dd88'

trace1 = go.Scatter(x = deaths_by_date.index, y = deaths_by_date['count'], name='Deaths by Date', line = dict( color = color1))
trace2 = go.Scatter(x = symptoms_by_date.index, y = symptoms_by_date['count'], name='Symptoms by Date', line = dict(  color = color2 ) )
trace3 = go.Scatter(x = entry_by_date.index, y = entry_by_date['count'], name='Entry by Date', line = dict(color = color3 ) )

data = [trace1,trace2,trace3]
layout = go.Layout(title= "Cases by day (Jan- Jun 2020)", yaxis=dict(title='Number of deaths', titlefont=dict(color=colort), tickfont=dict(color=colort)))
fig = go.Figure(data=data, layout=layout)
plot_url = py.iplot(fig)

<font size="+1" color="darkred">Insights: 
* Peaks and downs in symptoms and hospital admissions are in sync (very intuitive). 
* Deaths have been peaking from March and in between small dips and highest number of deaths on June 10. (93 deaths)
</font>

In [ ]:
sdates_df = icu_yes.groupby(['died_symptoms_dates']).agg('count')['id'].to_frame(name='count').reset_index()
entry_bool = sdates_df['died_symptoms_dates'] >= 0
sdates_df = sdates_df[entry_bool]
fig = px.scatter(sdates_df, x="died_symptoms_dates", y="count",
                 color="count", color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title="Number of days patients lived from Symptoms to Death ",title_x=0.5,xaxis=dict(title="Days"),yaxis=dict(title="Number of Patients"))
fig.show()

<font size="+1" color="darkred">Insights: 
* Around 26 patients died on same day that they noticed symptoms and got admitted. This seems strange. It could be they mistook the underlying conditions that escalated it.
* Patients who died after suffering a month is below 30. This could be because they had milder virus and the medical attention over weeks was working well to bring down deaths.
* There are around 248 patients who died in 8 days delta (difference between death and symptoms). Between 4 - 16 days (difference between death and symptoms), the number of patients who died sees a peak (up from around 150 - 250 patients). It is likely that medical care given was not proper or there were some other conditions that could not salvage the death. 
</font>

In [ ]:
ddates_df = icu_yes.groupby(['died_entry_dates']).agg('count')['id'].to_frame(name='count').reset_index()
entry_bool = ddates_df['died_entry_dates'] >= 0
ddates_df = ddates_df[entry_bool]
fig = px.scatter(ddates_df, x="died_entry_dates", y="count",
                 color="count", color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title="Number of days patients lived between Admission to Death ",title_x=0.5,xaxis=dict(title="Days"),yaxis=dict(title="Number of Patients"))
fig.show()

<font size="+1" color="darkred">Insights: 
* There are 326 patients died on same day that they got admitted. This seems strange. It is likely that they came in late and they could have come earlier.
* There are around 406 people who died a day later, this seems a situation where they had co-morbidities that spiralled out of control.
* We can see graph going down  meaning the longer they are in the hospital the number of patients who died is reducing. It is likely that medical care given was working and medical team is in control of the situation. 
</font>

In [ ]:
dates_df = icu_yes.groupby(['entry_symptoms_dates']).agg('count')['id'].to_frame(name='count').reset_index()
fig = px.scatter(dates_df, x="entry_symptoms_dates", y="count",
                 color="count", color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title="Number of days between - When Symptoms appeared to Hospital Admissions",title_x=0.5,xaxis=dict(title="Days"),yaxis=dict(title="Number of Patients"))
fig.show()

<font size="+1" color="darkred">Insights: 
* There are 1460 patients who got admitted on the same day they saw symptoms. The symptoms probably were very strong that they immediately sought a hospital admission.
* There are around 900 people who got admitted in a hospital everyday, from day 1 to day 7 of symptoms.
* We can see graph going down  meaning patients admitted within a week is high and admissions tapers down as the patients are dealing with symptoms longer- maybe on their own.
* After 2 weeks of symptoms, there has just been a case or two admissions in the hospitals. The virus must have been tackled within 2 weeks.
</font>

In [ ]:
negative_result_bool = (icu_yes['covid_res'] == 'Negative') 
positive_result_bool = (icu_yes['covid_res'] == 'Positive')
awaiting_result_bool = (icu_yes['covid_res'] == 'Awaiting Results') 
pne_bool = (icu_yes['pneumonia'] == 'Yes')
intubed_bool = (icu_yes['intubed'] == 'Yes')
copd_bool = (icu_yes['copd'] == 'Yes') 
tobacco_bool = (icu_yes['tobacco'] == 'Yes')
asthma_bool = (icu_yes['asthma'] == 'Yes')
sex_bool = (icu_yes['sex'] == 'Female') 
preg_bool = (icu_yes['pregnancy'] == 'Yes') 
awaiting_result_bool = (icu_yes['covid_res'] == 'Awaiting Results')
inmsupr_bool = (icu_yes['inmsupr'] == 'Yes')
obesity_bool = (icu_yes['obesity'] == 'Yes')
hypertension_bool = (icu_yes['hypertension'] == 'Yes')
cardiovascular_bool = (icu_yes['cardiovascular'] == 'Yes')
renal_chronic_bool = (icu_yes['renal_chronic'] == 'Yes')
other_disease_bool = (icu_yes['other_disease'] == 'Yes')
diabetes_bool = (icu_yes['diabetes'] == 'Yes')
non_diabetes_bool = (icu_yes['diabetes'] == 'Yes')
died_bool = icu_yes['died'] == True
not_died_bool = icu_yes['died'] == False

In [ ]:
fig = go.Figure()
x11 = icu_yes[positive_result_bool].shape[0]
x12 = icu_yes[positive_result_bool & pne_bool].shape[0]
x13 = icu_yes[positive_result_bool & pne_bool & intubed_bool].shape[0]
x14 = icu_yes[positive_result_bool & pne_bool & intubed_bool & tobacco_bool].shape[0]
x15 = icu_yes[positive_result_bool & pne_bool & intubed_bool & copd_bool & tobacco_bool].shape[0]
x16 = icu_yes[positive_result_bool & pne_bool & intubed_bool & asthma_bool & tobacco_bool & copd_bool].shape[0]

x21 = icu_yes[negative_result_bool].shape[0]
x22 = icu_yes[negative_result_bool & pne_bool].shape[0]
x23 = icu_yes[negative_result_bool & pne_bool & intubed_bool].shape[0]
x24 = icu_yes[negative_result_bool & pne_bool & intubed_bool & tobacco_bool].shape[0]
x25 = icu_yes[negative_result_bool & pne_bool & intubed_bool & copd_bool & tobacco_bool].shape[0]
x26 = icu_yes[negative_result_bool & pne_bool & intubed_bool & asthma_bool & tobacco_bool & copd_bool].shape[0]

x31 = icu_yes[awaiting_result_bool].shape[0]
x32 = icu_yes[awaiting_result_bool & pne_bool].shape[0]
x33 = icu_yes[awaiting_result_bool & pne_bool & intubed_bool].shape[0]
x34 = icu_yes[awaiting_result_bool & pne_bool & intubed_bool & tobacco_bool].shape[0]
x35 = icu_yes[awaiting_result_bool & pne_bool & intubed_bool & copd_bool & tobacco_bool].shape[0]
x36 = icu_yes[awaiting_result_bool & pne_bool & intubed_bool & asthma_bool & tobacco_bool & copd_bool].shape[0]

fig.add_trace(go.Funnel(
    name = 'Positive',
    y = ["Covid Result", "Pneumonia", "Intubed",'Tobacco',"COPD", 'Asthma'],
    x = [x11, x12, x13, x14, x15, x16], textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Negative', orientation = "h",
    y = ["Covid Result", "Pneumonia", "Intubed", 'Tobacco',"COPD", 'Asthma'],
    x = [x21, x22, x23, x24, x25, x26],
    textposition = "inside", textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'Awaiting Results/Unspecified', orientation = "h",
    y = ["Covid Result", "Pneumonia", "Intubed", 'Tobacco',"COPD", 'Asthma'], x = [x31, x32, x33, x34,x35, x36],
    textposition = "inside",
    textinfo = "value+percent previous"))
fig.update_layout(title="ICU Patients who have most common preconditions ",title_x=0.35,xaxis=dict(title="Number of Patients"),yaxis=dict(title="Preconditions"))
fig.show()

<font size="+1" color="darkred">Insights: 
* Let us look at Lung related diseases or infections because corona virus is known to decapacitate the lungs.
* There were 10112 ICU patients. 5822 patients are COVID +, 3187 patients are COVID- and 1103 patients are awaiting results.
* There are around 88% of COVID positive have pneumonia, similarly 75% of COVID negative have pneumonia and 80% of COVID Awaiting results have pneumonia. 
* Around 50% of above pneumonia positive people needed intubation.
* Around 4% of the above people who needed intubation, use tobacco.
* Around .4% of the above people who use tobacco have COPD issues.
* A very miniscule percentage of people who have COPD issues have asthma.
* So net-net, Pneumonia appears like a strong reason to need ICU whether the COVID result is positive or negative
</font>

In [ ]:
fig = go.Figure()

x21 = icu_yes[negative_result_bool].shape[0]
x22 = icu_yes[negative_result_bool & ~pne_bool].shape[0]
x23 = icu_yes[negative_result_bool & ~pne_bool & ~intubed_bool].shape[0]
x24 = icu_yes[negative_result_bool & ~pne_bool & ~intubed_bool & ~hypertension_bool].shape[0]
x25 = icu_yes[negative_result_bool & ~pne_bool & ~intubed_bool  & ~hypertension_bool & ~diabetes_bool].shape[0]
x26 = icu_yes[negative_result_bool & ~pne_bool & ~intubed_bool  & ~hypertension_bool & ~diabetes_bool & ~obesity_bool].shape[0]

x31 = icu_yes[awaiting_result_bool].shape[0]
x32 = icu_yes[awaiting_result_bool & ~pne_bool].shape[0]
x33 = icu_yes[awaiting_result_bool & ~pne_bool & ~intubed_bool].shape[0]
x34 = icu_yes[awaiting_result_bool & ~pne_bool & ~intubed_bool & ~hypertension_bool].shape[0]
x35 = icu_yes[awaiting_result_bool & ~pne_bool & ~intubed_bool & ~hypertension_bool & ~diabetes_bool ].shape[0]
x36 = icu_yes[awaiting_result_bool & ~pne_bool & ~intubed_bool & ~hypertension_bool & ~diabetes_bool & ~obesity_bool].shape[0]


fig.add_trace(go.Funnel(
    name = 'Negative', orientation = "h",
    y = ["Covid Results - Negative/ Awaiting Results", "No Pneumonia", "No Intubation Required", 'No hypertension','No diabetes',"No obesity"],
    x = [x21, x22, x23, x24, x25, x26],
    textposition = "inside",
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'Awaiting Results', orientation = "h",
    y = ["Covid Results - Negative/ Awaiting Results", "No Pneumonia", "No Intubation Required", 'No hypertension','No diabetes',"No obesity"],
    x = [x31, x32, x33, x34,x35, x36],
    textposition = "inside",
    textinfo = "value+percent previous"))
fig.update_layout(title="ICU Patients who did not have usual preconditions ",title_x=0.35,xaxis=dict(title="Number of Patients"),yaxis=dict(title="Preconditions"))

fig.show()

<font size="+1" color="darkred">Insights: 
* Let us look at ICU patients who did not have usual conditions.
* There were 10112 ICU patients. 5822 patients are COVID +, 3187 patients are COVID- and 1103 patients are awaiting results.
* Around 25% of COVID negative have no pneumonia (primary reason to have an ICU admission), similarly 20% of COVID awaiting results have no pneumonia.
* Around 80% of above pneumonia negative people needed NO intubation.
* Around 76% of the above people who needed no intubation, have no hypertension.
* Around 90% of the above people who have no hypertension have no diabetes.
* Around 93% of the above people who have no diabetes have no obesity either.
* So net-net, 411 patients who tested covid negative and 113 patients who results are awaiting - are free of usual suspects (preconditions /co-morbidities)
* So above 524 patients - we have no explanation at all. They might have had just low level of immunity to get affected by covid and needed an ICU.
* So, what circumstances create a need for ICU ?
</font>

In [ ]:
print("Printing the pre-condition of patient numbers: ")
print("Pnemonia patients: ",icu_yes[pne_bool].shape[0])
print("Non Pnemonia patients: ",icu_yes[~pne_bool].shape[0])
print("Percentage of Pnemonia patients: ",(icu_yes[pne_bool].shape[0]/(icu_yes[pne_bool].shape[0] + icu_yes[~pne_bool].shape[0]) * 100))
print("Intubed patients: ",icu_yes[intubed_bool].shape[0])
print("Non Intubed patients: ",icu_yes[~intubed_bool].shape[0])
print("Percentage of Intubed patients: ",(icu_yes[intubed_bool].shape[0]/(icu_yes[intubed_bool].shape[0] + icu_yes[~intubed_bool].shape[0]) * 100))
print("COPD patients: ",icu_yes[copd_bool].shape[0])
print("Non COPD patients: ",icu_yes[~copd_bool].shape[0])
print("Percentage of COPD patients: ",(icu_yes[copd_bool].shape[0]/(icu_yes[copd_bool].shape[0] + icu_yes[~copd_bool].shape[0]) * 100))
print("Asthma patients: ",icu_yes[asthma_bool].shape[0])
print("Non Asthma patients: ",icu_yes[~asthma_bool].shape[0])
print("Percentage of Asthma patients: ",(icu_yes[asthma_bool].shape[0]/(icu_yes[asthma_bool].shape[0] + icu_yes[~asthma_bool].shape[0]) * 100))
print("Tobacco patients: ",icu_yes[tobacco_bool].shape[0])
print("Non Tobacco patients: ",icu_yes[~tobacco_bool].shape[0])
print("Percentage of Tobacco patients: ",(icu_yes[tobacco_bool].shape[0]/(icu_yes[tobacco_bool].shape[0] + icu_yes[~tobacco_bool].shape[0]) * 100))
print("Immuno Supression patients: ",icu_yes[inmsupr_bool].shape[0])
print("Non Immuno Supression patients: ",icu_yes[~inmsupr_bool].shape[0])
print("Percentage of Immuno Supression patients: ",(icu_yes[inmsupr_bool].shape[0]/(icu_yes[inmsupr_bool].shape[0] + icu_yes[~inmsupr_bool].shape[0]) * 100))
print("Hypertension patients: ",icu_yes[hypertension_bool].shape[0])
print("Non Hypertension patients: ",icu_yes[~hypertension_bool].shape[0])
print("Percentage of Hypertension patients: ",(icu_yes[hypertension_bool].shape[0]/(icu_yes[hypertension_bool].shape[0] + icu_yes[~hypertension_bool].shape[0]) * 100))
print("Cardiovascular patients: ",icu_yes[cardiovascular_bool].shape[0])
print("Non Cardiovascular patients: ",icu_yes[~cardiovascular_bool].shape[0])
print("Percentage of Cardiovascular patients: ",(icu_yes[cardiovascular_bool].shape[0]/(icu_yes[cardiovascular_bool].shape[0] + icu_yes[~cardiovascular_bool].shape[0]) * 100))
print("Obesity patients: ",icu_yes[obesity_bool].shape[0])
print("Non Obesity patients: ",icu_yes[~obesity_bool].shape[0])
print("Percentage of Obesity patients: ",(icu_yes[obesity_bool].shape[0]/(icu_yes[obesity_bool].shape[0] + icu_yes[~obesity_bool].shape[0]) * 100))
print("Renal Chronic patients: ",icu_yes[renal_chronic_bool].shape[0])
print("Non Renal Chronic patients: ",icu_yes[~renal_chronic_bool].shape[0])
print("Percentage of Renal Chronic patients: ",(icu_yes[renal_chronic_bool].shape[0]/(icu_yes[renal_chronic_bool].shape[0] + icu_yes[~renal_chronic_bool].shape[0]) * 100))
print("Other disease patients: ",icu_yes[other_disease_bool].shape[0])
print("Non Other disease patients: ",icu_yes[~other_disease_bool].shape[0])
print("Percentage of Other disease_bool patients: ",(icu_yes[other_disease_bool].shape[0]/(icu_yes[other_disease_bool].shape[0] + icu_yes[~other_disease_bool].shape[0]) * 100))
print("Diabetes patients: ",icu_yes[diabetes_bool].shape[0])
print("Non Diabetes patients: ",icu_yes[~diabetes_bool].shape[0])
print("Percentage of Diabetes patients: ",(icu_yes[diabetes_bool].shape[0]/(icu_yes[diabetes_bool].shape[0] + icu_yes[~diabetes_bool].shape[0]) * 100))

In [ ]:
tree_df=icu_yes.groupby(['covid_res','pneumonia' ,'intubed','hypertension','diabetes','obesity','tobacco','cardiovascular','renal_chronic'])['id'].agg('count').to_frame(name='count').rename(columns={'id':'count'}).reset_index().sort_values(by='count',ascending=False)
tree_df

In [ ]:
tree_df["all"] = "All ICU Patients" # in order to have a single root node
fig = px.treemap(tree_df, path=['all', 'covid_res', 'pneumonia', 'intubed','hypertension','diabetes','obesity'], values='count', width=1600, height=900)#, color='pneumonia', color_continuous_midpoint=100, color_continuous_scale=px.colors.diverging.Portland)
fig.update_layout(treemapcolorway = ["brown", "magenta"])
fig.show()

<font size="+1" color="darkred">Insights:     

* Treemap takes all ICU patients and branches into covid results, then pneumonia, intubation, hypertension, diabetes, obesity
* There are 285 covid positive patients but no other above major health conditions. So why did they need an ICU ?
* There are 409 covid negative patients and no other above major health conditions. So why did they need an ICU ?
* There are 122 covid test results awaiting patients but no other above major health conditions. So why did they need an ICU ?
* It is possible some information is not recorded.
</font>

In [ ]:
def categorical_df(disease):
    print(icu_yes.groupby(['covid_res', disease, 'died' ])['id'].agg('count'))
    categorical_df = icu_yes.groupby(['covid_res',disease,'died'])['id'].agg('count').to_frame(name='count').rename(columns={'id':'count'}).reset_index().sort_values(by='count',ascending=False)    
    return categorical_df

disease = categorical_df('inmsupr')
sns.catplot(x="covid_res", y="count", hue="died", col='inmsupr',aspect=.65, kind="swarm", data=disease)

<font size="+1" color="darkred">Insights: 
    
* If you have an autoimmune disorder, you probably take medicine to manage your condition. Some of those drugs can raise your chances of getting an infection. That might include COVID-19, the disease caused by the new coronavirus. And medicines called immunosuppressants may make you more likely to have serious complications from the virus, as can your autoimmune disorder itself.

But you shouldn’t stop taking your medicine on your own. Instead, talk to your doctor about your concerns and whether you should adjust your treatment.
* Let us look at ICU patients whose covid results were positive, negative and awaiting results.
* There were 10112 ICU patients. 5822 patients are COVID +, 3187 patients are COVID- and 1103 patients are awaiting results.
* As per 2nd graph, patients who took immunosupressant and were COVID positive is very low
* As per 1st graph, patients who did not take immunosupressants and were COVID positive is very high.
* As per the analysis we can conclude immunosuppressants did not play a role here.
* So, we can dismiss this as a side-reason for needing a ICU.
</font>

In [ ]:
# Create dimensions
other_disease_dim = go.parcats.Dimension(values=icu_yes.other_disease, categoryorder='category descending', label="Any Other Diseases")
contact_dim = go.parcats.Dimension(values=icu_yes.contact_other_covid, categoryorder='category descending',label="Contact Other Covid Patients")
survival_dim = go.parcats.Dimension(values=icu_yes.died, categoryorder='category descending',label="Died" )

# Create parcats trace
color = icu_yes.died;
colorscale = [[0, 'olive'], [0.13, 'violet'], [0.23, 'deeppink'],[0.33, 'firebrick'], [0.66, 'turquoise'], [0.66, 'seagreen'], [1.0, 'powderblue']]#[[0, 'lightsteelblue'], [1, 'firebrick']];

fig = go.Figure(data = [go.Parcats(dimensions=[other_disease_dim, contact_dim, survival_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 22, 'family': 'Raleway'},
        tickfont={'size': 18, 'family': 'Raleway'},
        arrangement='freeform')])

fig.show()

<font size="+1" color="darkred">Insights: 
* Around 628 patients who died had a contact with other covid patients, but no other diseases.
* Around 2139 patients who died had no contact with other covid patients and also no other diseases.
</font>

<a id="gender"></a>
<font size="+2" color="blue">Analysis of Covid Results across gender</font>

In [ ]:
gender_df=icu_yes.groupby(['sex','covid_res']).agg('count')['id'].to_frame(name='count').reset_index()
gender_male=gender_df.loc[gender_df['sex']=='Male']
gender_female=gender_df.loc[gender_df['sex']=='Female']

male=go.Bar(x=gender_male['covid_res'],y=gender_male['count'],marker=dict(color='brown'),name="male")
female=go.Bar(x=gender_female['covid_res'],y=gender_female['count'],marker=dict(color='orange'),name="female")
data=[male,female]

fig = go.Figure(data)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_layout(title="Covid Results - Gender (Jan-Jun 2020)",title_x=0.5,xaxis=dict(title="Covid Results"),yaxis=dict(title="Count"),
                   barmode="group")
fig.show()

<font size="+1" color="darkred">Insights: 
    
* Let us look at ICU patients whose covid results were positive, negative and awaiting results.
* There were 10112 ICU patients. 5822 patients are COVID +, 3187 patients are COVID- and 1103 patients are awaiting results.
* COVID +ve --> Male = 3905, Female = 1917 (Around 49%)
* COVID -ve  --> Male = 1899, Female = 1288 (Around 67%)
* There are more male patients than female patients who are positive.
* We cannot conclude that men are more susceptible right away. 
* In some cultures, men work outside and women are encouraged to work indoors or stay at home.    
</font>

<a id="age"></a>
<font size="+2" color="blue">Analysis of age across age-frequency, gender</font>

In [ ]:
male_hist_data = icu_yes[icu_yes['sex']=='Male']['age']
female_hist_data = icu_yes[icu_yes['sex']=='Female']['age']
group_labels = ['Male','Female'] 

hist_data = [male_hist_data, female_hist_data]
fig=go.Figure()
fig=ff.create_distplot(hist_data, group_labels,bin_size=12)
fig.update_layout(title_text="Distribution of Age - Gender wise",title_x=0.5)
fig.show()

<font size="+1" color="darkred">Insights: 
    
* In age range between [0-11],[12-23],[24-35] years, the female patients who needed ICU are more than the male patients who needed ICU.
* In age ranges between [35 - 70] years, the male patients who needed ICU is more than the female patients who needed ICU.
* In age ranges between [71-onwards], the female patients need ICU more than male patients.
</font>

In [ ]:
temp = pd.DataFrame({'age_freq':icu_yes.age_freq.value_counts()})
df = temp
df = df.sort_values(by='age_freq', ascending=True)
data  = go.Data([
            go.Bar(
              y = df.index,
              x = df.age_freq,
              orientation='h'
        )])
layout = go.Layout(
       
        margin=go.layout.Margin(l=300),
        title = "Number of Patients in various age groups"
)
fig  = go.Figure(data=data, layout=layout)
py.iplot(fig)

<font size="+1" color="darkred">Insights: 
    
* In age range between [51 - 70] years, the patients who needed ICU is 4130 .
* In age ranges between [31 - 50] years, the patients who needed ICU is 2710.
* In age ranges between [71-onwards], the number is 1690.
* Infants under age two, the number is 652.  
* Youth between [18-30] years, the number is 585.
* Children between [2-12] years, the number is 243.
* Teenagers between [13-17] years, the number is 102.
    
* Combining the analyses, a male patient between 31-70 years is more likely to need and ICU than a female counterpart and the numbers speak so
* A female patient whose age is above 70 or female infants and female children[2-17] years and female youth[18-30] years patients are more likely to need an ICU than male counterparts.    
</font>

In [ ]:
fig = px.parallel_categories(icu_yes, dimensions=['died','covid_res','sex'],
                color="age", color_continuous_scale=px.colors.sequential.Inferno, # Color for Pclass
               labels={'survived_or_not':'Died', 'covid_res':'Covid Result'}) # labeling
fig.update_layout(title="ICU Parallel Categories Diagram ")
fig.show()

<font size="+1" color="darkred">Insights: 
    
* 3948 patients out of 10112 ICU patients died. Around 39% of total patients died.
* Out of these 3948 patients, around 767 patients covid results came negative. Either tests were not good enough or their treatment for some other conditions failed.
  
</font>

In [ ]:
print(icu_yes['died'].value_counts())
icu_yes.groupby(['died','covid_res','sex']).agg('count')

In [ ]:
headerColor = 'red'
rowEvenColor = 'lightpink'
rowOddColor = 'skyblue'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>CONDITIONS</b>','<b>YES</b>','<b>NO</b>','<b>HAVING CONDITION AND DIED</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=16)
  ),
  cells=dict(
    values=[
      ['Pneumonia', 'Intubed', 'COPD', 'Asthma', 'Tobacco', 'Immuno Supression taken','Hypertension','Cardiovascular Disease','Obesity','Renal Chronic Disease','Diabetes','Other Diseases'],
      [icu_yes[pne_bool].shape[0], icu_yes[intubed_bool].shape[0], icu_yes[copd_bool].shape[0], icu_yes[asthma_bool].shape[0], icu_yes[tobacco_bool].shape[0], icu_yes[inmsupr_bool].shape[0], icu_yes[hypertension_bool].shape[0], icu_yes[cardiovascular_bool].shape[0], icu_yes[obesity_bool].shape[0],icu_yes[renal_chronic_bool].shape[0],icu_yes[diabetes_bool].shape[0], icu_yes[other_disease_bool].shape[0]] ,
      [icu_yes[~pne_bool].shape[0], icu_yes[~intubed_bool].shape[0], icu_yes[~copd_bool].shape[0], icu_yes[~asthma_bool].shape[0], icu_yes[~tobacco_bool].shape[0], icu_yes[~inmsupr_bool].shape[0], icu_yes[~hypertension_bool].shape[0],icu_yes[~cardiovascular_bool].shape[0], icu_yes[~obesity_bool].shape[0], icu_yes[~renal_chronic_bool].shape[0], icu_yes[~diabetes_bool].shape[0], icu_yes[~other_disease_bool].shape[0]  ],
      [icu_yes[pne_bool & died_bool].shape[0], icu_yes[intubed_bool & died_bool].shape[0], icu_yes[copd_bool & died_bool].shape[0], icu_yes[asthma_bool & died_bool].shape[0], icu_yes[tobacco_bool & died_bool].shape[0], icu_yes[inmsupr_bool & died_bool].shape[0], icu_yes[hypertension_bool & died_bool].shape[0], icu_yes[cardiovascular_bool & died_bool].shape[0], icu_yes[obesity_bool & died_bool].shape[0],icu_yes[renal_chronic_bool & died_bool].shape[0],icu_yes[diabetes_bool & died_bool].shape[0], icu_yes[other_disease_bool & died_bool].shape[0]   ]   ],
      line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor,rowEvenColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'black', size = 14)
    ))
])

fig.show()

<font size="+1" color="darkred">Insights: 
    
* 'YES' column indicates the number of patients having the specified condition, while the last column indicates the number of patients who were having the specified condition and succumbed.
* It is likely patients have one or more diseases or pre-conditions.
  
</font>

In [ ]:
 
print("="* 90)
preg_df = icu_yes[sex_bool & preg_bool]
non_preg_df = icu_yes[sex_bool & ~preg_bool]
print("Women who needed ICU, pregnant is {} ".format(preg_df.shape[0]))
print("Women who needed ICU, non pregnant is {} ".format(icu_yes[sex_bool & ~preg_bool].shape[0]))

<font size="+1" color="darkred">Insights: 
    
* There are 74 pregnant female patients out of total 3600+ female patients.
  
</font>